In [1]:
import pandas as pd
import re

# KR operations

In [2]:
def extract_locality_and_street(input_string):
    # Список подстрок, которые могут предшествовать названию населенного пункта
    prefixes = ["г.", "г ", "с.", "с ", "кп ", "к.п.", "Город ", "город "]
    locality_keyword = ":"

    # Начальные значения для города и улицы
    locality = ""
    street = input_string

    locality_index = input_string.find(locality_keyword)
    input_string = input_string[locality_index:]

    # Поиск подстроки-префикса
    for prefix in prefixes:
        index = input_string.find(prefix)
        if index != -1:
            comma_index = input_string.find(',', index)
            locality = input_string[index:comma_index].strip()
            street_start = comma_index + 1
            street = input_string[street_start:].strip()
            break

    return locality, street

In [3]:
KR_files = ["KR on 01.01.2017 unprotected.xlsx", "KR on 01.01.2018 unprotected.xlsx", "KR on 01.01.2019 unprotected.xlsx"]

results = []

for KR_file in KR_files:
    filexlsx = pd.read_excel("train_data/KC_TR_KR/"+KR_file, skiprows=5) # Skip header

    columns_to_drop = [0, 2, 4, 6, 7, 10]
    filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)

    new_column_names = ["name", "estimated_cost", "approved_year_start", "used_year_end", "expected_year_end"]
    filexlsx.columns = new_column_names

    target_line_startswith = ["Выборочный", "Капитальный", "Новый", "Ремонт"]
    filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]

    filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

    results.append(filtered_rows)

/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_34299/3769734642.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_34299/3769734642.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders

In [4]:
results[0]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
5,Капитальный ремонт подземного сооружения (Лит....,4088.43,4077.25,3927.48,3927.48,г. Владивосток,"ул. Светланская, д. 71"
8,"Капитальный ремонт здания РКЦ Находка, располо...",19001.91,1149.83,1143.30,1143.30,г. Находка,"проспект Находкинский, 13"
13,Капитальный ремонт гаража по адресу: Амурская ...,13677.35,12858.01,12858.01,12858.01,г. Благовещенск,"пер. Святителя Иннокентия, 17"
16,Выборочный капитальный ремонт административног...,18034.12,1298.34,1298.33,1298.33,г. Тында,"ул. Профсоюзная, 12"
19,Выборочный капитальный ремонт административног...,448.72,478.98,448.72,448.72,г. Благовещенск,"пер. Св. Иннокентия, 17"
24,Выборочный капитальный ремонт здания гаража на...,4439.86,199.79,199.79,199.79,г. Петропавловск-Камчатский,"ул. Ломоносова, 2"
29,Выборочный капитальный ремонт сетей теплоснабж...,97.37,97.37,97.37,97.37,г. Магадан,"ул. Пролетарская, д. 8, строение 1, литера А, ..."
34,Выборочный капитальный ремонт административног...,2344.49,3281.74,2344.49,2344.49,г. Покровск,"ул. Орджоникидзе, д. 24"
39,"Выборочный капитальный ремонт ""Сооружения комм...",184.94,184.95,184.94,184.94,,"Выборочный капитальный ремонт ""Сооружения комм..."
42,Выборочный капитальный ремонт в части усиления...,3634.45,3621.51,3308.46,3308.46,г. Южно-Сахалинск,"Коммунистический проспект, 47 (литер А)"


In [5]:
results[1]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
5,Капитальный ремонт помещений амбулатории и кро...,10943.56,670,670.00,670.00,г. Владивосток,"ул. Светланская, 71"
8,"Капитальный ремонт здания РКЦ Находка, располо...",21443.21,8817.68,8817.67,8817.67,г. Находка,"проспект Находкинский, д. 13"
11,Выборочный капитальный ремонт ограждения и вор...,5349.85,5377.36,5349.85,5349.85,г. Уссурийск,"ул. Некрасова, 102"
14,Капитальный ремонт водопровода в гараже на 10 ...,283.66,283.66,283.66,283.66,г. Уссурийск,"ул. Некрасова, д. 102"
17,Новый объект. Выборочный капитальный ремонт п...,154.41,154.42,154.41,154.41,г. Уссурийск,"ул. Некрасова, 102"
22,Выборочный капитальный ремонт помещений 3 этаж...,19875.80,21461.5,19875.80,19875.80,г. Хабаровск,"ул. Муравьева-Амурского, 21"
25,Выборочный капитальный ремонт колодца ВК1 объе...,176.18,196.5,176.18,176.18,г. Хабаровск,"ул. Калинина, 156"
30,Капитальный ремонт кровли блока А здания по ад...,2506.52,2506.52,2506.52,2506.52,г. Петропавловск-Камчатский,"просп. К. Маркса, 29/2"
35,Выборочный капитальный ремонт первого этажа во...,2999.90,2999.9,2999.90,2999.90,г. Южно-Сахалинск,"Коммунистический проспект, 47"
38,"Выборочный капитальный ремонт объекта ""Сооруже...",1699.56,1699.56,849.78,849.78,,"Выборочный капитальный ремонт объекта ""Сооруже..."


In [6]:
results[2]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
5,Капитальный ремонт помещений и кровли в пятиэт...,670.00,9170.91,9170.90,12.2018,г. Владивосток,"ул. Светланская, 71"
8,"Капитальный ремонт здания РКЦ Находка, располо...",9960.98,11482.32,11482.32,5.2018,г.Находка,"проспект Находкинский, д. 13"
11,"Капитальный ремонт здания, расположенного по а...",NaN,375.72,375.71,10.2019,г.Владивосток,"ул. Калинина, 261"
14,Капитальный ремонт внешнего электроснабжения з...,NaN,290,290.00,6.2020,г. Владивосток,"ул. Светланская, д.71, д.73, д.73а"
17,"Капитальный ремонт нежилых помещений (склада),...",NaN,226.46,226.45,9.2018,г.Владивосток,"ул.Фонтанная, 19"
20,Капитальный ремонт инженерных сетей на объекта...,NaN,1227.79,1227.79,7.2018,г. Владивосток,"ул. Пушкинская, 8б"
25,Выборочный капитальный ремонт помещений 2 эта...,NaN,156.65,0.00,8.2019,г. Хабаровск,"ул. Муравьева-Амурского,21"
28,Выборочный капитальный ремонт аварийных участк...,NaN,175.56,140.46,12.2018,г. Хабаровск,участок находится примерно в 166 м по направле...
33,Выборочный капитальный ремонт здания гаража на...,199.79,3397.65,3397.65,8.2018,г. Петропавловск-Камчатский,"ул. Ломоносова, 2"
36,Выборочный капитальный ремонт системы хозяйств...,NaN,490.53,490.53,8.2018,г. Петропавловск-Камчатский,"просп. К.Маркса, 29/2"


## New version sheets

In [7]:
KR_files = ["KR on 01.01.2020.xlsx", "KR on 01.01.2021.xlsx", "KR on 01.01.2022.xlsx"]

for KR_file in KR_files:
    filexlsx = pd.read_excel("train_data/KC_TR_KR/"+KR_file, skiprows=2) # Skip header

    columns_to_drop = [0, 2, 4, 5, 7, 9, 10, 12, 13, 14, 15, 16, 17]
    filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)

    new_column_names = ["name", "estimated_cost", "approved_year_start", "used_year_end", "expected_year_end"]
    filexlsx.columns = new_column_names

    ### Old version sheets were in thousands roubles 
    columns_to_divide = ['estimated_cost', 'approved_year_start', 'used_year_end', 'expected_year_end']
    # Convert the specified columns to numeric type
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].apply(pd.to_numeric, errors='coerce')
    # Divide
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].div(1000)
    # Round the numeric values to two decimal places
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].round(2)
    # Convert the columns back to string type
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].astype(str)

    target_line_startswith = ["Выборочный", "Капитальный", "Новый", "Ремонт"]
    filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]

    filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

    results.append(filtered_rows)


/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_34299/4001036180.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_34299/4001036180.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders

In [8]:
results[3]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Капитальный ремонт здания убежища (лит. А) Дал...,9437.31,1800.13,0.0,0.0,г. Владивосток,"ул. Семеновская, 29а"
14,Капитальный ремонт внешнего электроснабжения з...,6706.61,6416.61,6416.61,6416.61,г. Владивосток,"ул. Светланская, д.71, д.73, д.73а"
17,"Капитальный ремонт здания, расположенного по а...",83550.9,1469.48,1469.48,1469.48,г. Владивосток,"ул. Светланская, 71"
20,Капитальный ремонт нежилых помещений и помещен...,30232.72,1653.67,0.0,0.0,г. Владивосток,"ул. Светланская, д.73"
23,Выборочный капитальный ремонт помещения бокс-...,12529.12,1429.23,0.0,0.0,г. Владивосток,"ул. Светланская, 71"
26,"Капитальный ремонт здания, расположенного по а...",4529.89,1653.94,1654.43,1654.43,с. Вольно-Надеждинское,"ул. Пушкина, д. 59"
29,"Выборочный капитальный ремонт РКЦ, пристройки ...",15492.99,14736.19,0.0,0.0,г. Уссурийск,"ул. Некрасова, д.102"
32,Выборочный капитальный ремонт гаража РКЦ на 2 ...,2014.98,1877.47,908.62,908.62,г. Уссурийск,"ул. Некрасова, д.102"
35,Выборочный капитальный ремонт нежилых помещен...,2875.63,263.6,0.0,0.0,г. Владивосток,"ул. Светланская, д.73"
38,Выборочный капитальный ремонт здания Централь...,3942.55,258.0,258.0,258.0,г. Владивосток,"ул. Океанский проспект, д.34"


In [9]:
results[4]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Выборочный капитальный ремонт здания Централь...,3942.55,1836.16,0.0,0.0,г. Владивосток,"ул. Океанский проспект, д.34"
14,Выборочный капитальный ремонт здания Централь...,97.78,97.78,97.78,97.78,г. Владивосток,"Океанский проспект, д.34 в части замены дверно..."
17,Выборочный капитальный ремонт нежилых помещен...,2907.5,2907.5,235.59,235.59,г. Владивосток,"ул. Светланская, д.73"
20,Выборочный капитальный ремонт помещения бокс-...,12784.08,12309.38,2308.49,2308.49,г. Владивосток,"ул. Светланская, 71"
23,Капитальный ремонт здания убежища (лит. А) Дал...,9465.23,9163.94,9198.95,9198.95,г. Владивосток,"ул. Семеновская, 29а"
26,Капитальный ремонт нежилых помещений и помещен...,31288.37,30776.85,15126.51,15126.51,г. Владивосток,"ул. Светланская, д.73"
29,"Капитальный ремонт здания, расположенного по а...",4924.75,2981.81,2981.31,2981.31,с. Вольно-Надеждинское,"ул. Пушкина, д. 59"
32,"Выборочный капитальный ремонт РКЦ, пристройки ...",16313.17,15556.36,5325.76,5325.76,г. Уссурийск,"ул. Некрасова, д.102"
35,Выборочный капитальный ремонт гаража РКЦ на 2 ...,1946.33,895.2,895.2,895.2,г. Уссурийск,"ул. Некрасова, д.102"
38,Выборочный капитальный ремонт гаража РКЦ на 10...,1534.44,1483.53,1483.53,1483.53,г. Уссурийск,"ул. Некрасова, д.102"


In [10]:
results[5]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Выборочный капитальный ремонт здания Централь...,4415.99,4157.99,3871.0,3871.0,г. Владивосток,"ул. Океанский проспект, д.34"
14,Выборочный капитальный ремонт нежилого здания ...,4243.68,4243.68,139.45,139.45,г. о. Владивостокский,"г. Владивосток, пр-кт Океанский, д. 34 в части..."
17,Выборочный капитальный ремонт помещения бокс-...,12475.7,9692.51,3909.53,3909.53,г. Владивосток,"ул. Светланская, 71"
20,Выборочный капитальный ремонт нежилых помещени...,2907.5,2671.9,945.59,945.59,г. Владивосток,"ул. Светланская, д.73"
23,Капитальный ремонт нежилых помещений и помещен...,31288.37,15650.33,11776.82,11776.82,г. Владивосток,"ул. Светланская, д.73"
26,Выборочный капитальный ремонт нежилого здания ...,4074.53,153.69,153.69,153.69,город Уссурийск,"улица Некрасова, здание № 102 в части замены л..."
29,"Выборочный капитальный ремонт РКЦ, пристройки ...",16107.76,1212.79,1212.79,1212.79,г. Уссурийск,"ул. Некрасова, д.102"
32,Выборочный капитальный ремонт с созданием авто...,57.28,57.28,57.28,57.28,г.о. Владивостокский,"г. Владивосток, пр-кт Океанский, д. 34"
35,Выборочный капитальный ремонт помещения № 14 в...,1212.9,1212.9,1212.9,1212.9,г.Владивосток ГУ Банка России по Приморскому к...,"Российская Федерация, Приморский край, г. о. В..."
39,Капитальный ремонт с перепланировкой помещений...,43837.95,999.62,999.62,999.62,г город Благовещенск,"Город Благовещенск, Переулок Св. Иннокентия, Д..."
